In [1]:
import agent
from importlib import reload 
import cube_sim
# import eval
# reload(cube_sim)
# reload(agent)
import itertools
import numpy as np
import pandas as pd
from pathlib import Path
import os
import pickle
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib widget

In [2]:
a = agent.ApproximateQAgent()

In [20]:
a = a.load(fname='12_05_v1.1_episode_4000',inpath=Path('./output'))

Loading agent from output


In [4]:
a.weights

{'b': 15.93480349980175,
 'one_color_face': 31.770351443088096,
 'two_color_face': 16.564541155466934,
 'three_color_face': 32.98568029029012,
 'four_color_face': 14.088248109958462,
 'front_top': -10.652584711552745,
 'front_bottom': -10.566243331083879,
 'front_left': -5.593810156332125,
 'front_right': -11.152364020068356,
 'front_back': -9.603485127186138,
 'top_bottom': -18.49149417584755,
 'top_left': -8.364993492353275,
 'top_right': -14.1720562569074,
 'top_back': -8.914783569355345,
 'bottom_left': -10.737972241922922,
 'bottom_right': -14.597841523094774,
 'bottom_back': -8.573699047158522,
 'left_right': -11.274047478364361,
 'left_back': -7.806132554801266,
 'right_back': -6.966822551336041,
 'full_layers': 45.69084690288183}

In [5]:
epdf = pd.DataFrame.from_dict(a.metadata,orient='index')

In [8]:
epdf.tail()

,MovesToGoal,EpisodeRewards,Weights,TotalRunTime,EpisodeRunTime,EndEpsilon
ep_3995,999,11161.0,"{'b': 15.93480349980175, 'one_color_face': 31....",7680.935804,7.961296,0.1
ep_3996,999,14156.0,"{'b': 15.93480349980175, 'one_color_face': 31....",7688.982757,8.046953,0.1
ep_3997,999,6761.0,"{'b': 15.93480349980175, 'one_color_face': 31....",7696.937850,7.955094,0.1
ep_3998,999,8539.0,"{'b': 15.93480349980175, 'one_color_face': 31....",7704.784997,7.847147,0.1
ep_3999,999,11570.0,"{'b': 15.93480349980175, 'one_color_face': 31....",7712.758008,7.973011,0.1


In [18]:
plt.close("all")
x = epdf.index

rwdcolor = 'tab:blue'
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(x,epdf[['EpisodeRewards']].rolling(100).mean(), color=rwdcolor,label='Rewards')
ax.tick_params(axis='x',rotation=60)
ax.tick_params(axis='y', labelcolor=rwdcolor)
ax.set_xlim(left=80)
# ax.set_xlabel('Episode')  # Add an x-label to the axes.
ax.set_ylabel('Rewards', color=rwdcolor, fontsize=14)  # Add a y-label to the axes.
ax.set_title("Rewards Per Episode (Rolling Mean of Previous 100 Episodes)",fontsize=15)  # Add a title to the axes.
xtick = list(epdf.index[249::250])
xtick.append(epdf.index[-1])
xlab = [f'Ep {i[3:]}' for i in xtick]
ax.set_xticks([])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

epscolor = 'red'
ax2.set_ylabel('Epsilon Value', color=epscolor, fontsize=14)  # we already handled the x-label with ax1
ax2.plot(x, epdf['EndEpsilon'], color=epscolor,label="Epsilon",lw=1,ls='--')
ax2.tick_params(axis='y', labelcolor=epscolor)
ax.set_xticks(xtick)
ax.set_xticklabels(xlab)
# ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)

solved = epdf[['EpisodeRewards']].rolling(100).mean()[epdf.MovesToGoal < 999]

ax.plot(solved, color='tab:blue',label="Solved",ls='',marker='o',markerfacecolor='white',ms=5)

fig.legend(loc=(.75,.375))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Plot solved comparison

In [76]:
e_trained_100 = agent.evaluator(a).load(fname='trained_100_iter_5000',inpath=Path('./output'))
e_rand_100 = agent.evaluator(a).load(fname='random_100_iter_5000',inpath=Path('./output'))

e_trained_1000 = agent.evaluator(a).load(fname='trained_iter_1000',inpath=Path('./output'))
e_rand_1000 = agent.evaluator(a).load(fname='random_iter999',inpath=Path('./output'))

e_trained_10000 = agent.evaluator(a).load(fname='trained_10000_iter_300',inpath=Path('./output'))
e_rand_10000 = agent.evaluator(a).load(fname='random_10000_iter_300',inpath=Path('./output'))

Loading eval from output
Loading eval from output
Loading eval from output
Loading eval from output
Loading eval from output
Loading eval from output


In [77]:
dft100 = pd.DataFrame.from_dict(e_trained_100.eval_data,orient='index')
dfr100 = pd.DataFrame.from_dict(e_rand_100.eval_data,orient='index')

dft1000 = pd.DataFrame.from_dict(e_trained_1000.eval_data,orient='index')
dfr1000 = pd.DataFrame.from_dict(e_rand_1000.eval_data,orient='index')

dft10000 = pd.DataFrame.from_dict(e_trained_10000.eval_data,orient='index')
dfr10000 = pd.DataFrame.from_dict(e_rand_10000.eval_data,orient='index')

In [78]:
t100solved = dft100.Solved.sum() / dft100.Solved.shape[0]
r100solved = dfr100.Solved.sum() / dfr100.Solved.shape[0]

t1000solved = dft1000.Solved.sum() / dft1000.Solved.shape[0]
r1000solved = dfr1000.Solved.sum() / dfr1000.Solved.shape[0]

t10000solved = dft10000.Solved.sum() / dft10000.Solved.shape[0]
r10000solved = dfr10000.Solved.sum() / dfr10000.Solved.shape[0]

In [79]:
plt.close("all")

labels = ['100 Max','1000 Max', '10000 Max']
t = [t100solved,t1000solved,t10000solved]
r = [r100solved,r1000solved,r1000solved]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, t, width, label='Trained Agent')
rects2 = ax.bar(x + width/2, r, width, label='Random Shuffler')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Max Moves per Iteration')
ax.set_ylabel('Proportion Solved')
ax.set_title('Proportion of Iterations Solved')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …